In [126]:
from typing import Dict, Text
import pprint

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

import pandas as pd

import json

json_string = '{ "name":"John", "age":30, "car":"None" }'
your_json = json.loads(json_string)
print(your_json)

recipes = pd.read_csv("recipes.csv")
users = pd.read_csv("users.csv")

recipes = recipes.astype({"recipeId": str, "name": str})
users = users.astype({"userId": str, "recipeId": str, "name": str})

newColumn = []


# for i in recipes['categories']:
#     temp = i.split(" | ")
#     newColumn.append(temp)

# recipes.drop(columns=['categories'])
# recipes["categories"] = newColumn
# print(recipes)

# print(recipes.dtypes)

recipes = tf.data.Dataset.from_tensor_slices(dict(recipes))
users = tf.data.Dataset.from_tensor_slices(dict(users))

users = users.map(lambda x: {
    "name": x["name"],
    "userId": x["userId"]
})

recipes = recipes.map(lambda x: x["name"])
                      
# for x in users.take(1).as_numpy_iterator():
#   pprint.pprint(x)

user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(users.map(lambda x: x["userId"]))

recipe_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
recipe_titles_vocabulary.adapt(recipes)


class RecipeModel(tfrs.Model):

  def __init__(
      self,
      user_model: tf.keras.Model,
      recipe_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    self.user_model = user_model
    self.recipe_model = recipe_model
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    user_embeddings = self.user_model(features["userId"])
    positive_recipe_embeddings = self.recipe_model(features["name"])
    return self.task(user_embeddings, positive_recipe_embeddings)

user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])
recipe_model = tf.keras.Sequential([
    recipe_titles_vocabulary,
    tf.keras.layers.Embedding(recipe_titles_vocabulary.vocab_size(), 64)
])

task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    recipes.batch(128).map(recipe_model)
  )
)

model = RecipeModel(user_model, recipe_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

model.fit(users.batch(4096), epochs=5)

index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    recipes.batch(100).map(lambda title: (title, model.recipe_model(title))))

# Get some recommendations.

{'name': 'John', 'age': 30, 'car': 'None'}
{'name': b'Chilled Prune Whip', 'userId': b'3526'}
Epoch 1/5
3/3 [==============================] - 1s 112ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0012 - factorized_top_k/top_5_categorical_accuracy: 0.0077 - factorized_top_k/top_10_categorical_accuracy: 0.0181 - factorized_top_k/top_50_categorical_accuracy: 0.1186 - factorized_top_k/top_100_categorical_accuracy: 0.3409 - loss: 23860.1987 - regularization_loss: 0.0000e+00 - total_loss: 23860.1987      
Epoch 2/5
3/3 [==============================] - 0s 108ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0498 - factorized_top_k/top_5_categorical_accuracy: 0.2593 - factorized_top_k/top_10_categorical_accuracy: 0.3395 - factorized_top_k/top_50_categorical_accuracy: 0.6068 - factorized_top_k/top_100_categorical_accuracy: 0.7713 - loss: 22293.7939 - regularization_loss: 0.0000e+00 - total_loss: 22293.7939
Epoch 3/5
3/3 [==============================] - 0s 106ms/step - facto

In [127]:
final_recommendations = {}

for i in range(0, 9999):
    _, titles = index(np.array([str(i)]))
    temp = []
    for j in titles[0, :6]:
        temp.append(j.numpy().decode("utf-8"))
    final_recommendations[str(i)] = temp

In [128]:
app_json = json.dumps(final_recommendations)

In [130]:
with open("recommendations.json", "w") as outfile:
    outfile.write(app_json)